In [1]:
import numpy as np

from bayescache.data import P3B3
from bayescache.api.data import train_valid_split
from bayescache.api.data.source import TrainingData

from torch.utils.data import DataLoader

In [2]:
train = P3B3('/Users/yngtodd/data', partition='train', download=True)

In [3]:
_, y_train = train.load_data()

In [4]:
np.unique(y_train['histology'])

array([0, 1])

In [5]:
trainloader = DataLoader(train, batch_size=1)

In [6]:
for idx, (data, labels) in enumerate(trainloader):
    print(labels)
    if idx == 2:
        break

{'subsite': tensor([5]), 'laterality': tensor([0]), 'histology': tensor([1]), 'grade': tensor([2])}
{'subsite': tensor([4]), 'laterality': tensor([0]), 'histology': tensor([1]), 'grade': tensor([1])}
{'subsite': tensor([5]), 'laterality': tensor([0]), 'histology': tensor([0]), 'grade': tensor([0])}


In [ ]:
train, valid = train_valid_split(train, valpercent=.25)

In [ ]:
len(train)

In [ ]:
len(valid)

In [ ]:
traindata = TrainingData(train, .25, 2, 1)

In [ ]:
val_loader = traindata.val_loader()

In [ ]:
for idx, (data, label) in enumerate(val_loader):
    print(f'idx: {idx}, label: {label}')
    if idx == 2:
        break

In [ ]:
valid = traindata.val_dataset()

In [ ]:
len(valid)

In [ ]:
train = traindata.train_dataset()

In [ ]:
len(train)

In [ ]:
source = traindata.train_loader()

In [ ]:
for idx, (data, labels) in enumerate(source):
    print(f'idx: {idx}, label: {label}')
    if idx == 2:
        break

### Testing the Loss Functions

In [7]:
import torch.nn.functional as F
from bayescache.models import mtcnn

In [8]:
hparams = mtcnn.Hyperparameters()

In [9]:
model = mtcnn.new()

In [10]:
print(f'word dim: {hparams.word_dim}')
print(f'max sentence length: {hparams.max_sent_len}')
print(f'vocab size: {hparams.vocab_size}')

word dim: 300
max sentence length: 1500
vocab size: 35095


In [11]:
data.shape

torch.Size([1, 1500])

In [12]:
logits = model(data)

In [13]:
data.shape

torch.Size([1, 1500])

In [14]:
logits

{'subsite': tensor([[ 1.6773,  1.6992, -2.5772, -1.8696,  0.9741, -1.4765]],
        grad_fn=<AddmmBackward>),
 'laterality': tensor([[1.3283, 1.4390]], grad_fn=<AddmmBackward>),
 'histology': tensor([[-1.1773,  5.1979]], grad_fn=<AddmmBackward>),
 'grade': tensor([[-0.4902,  0.1122, -0.6902]], grad_fn=<AddmmBackward>)}

In [15]:
loss = model.loss_value(data, labels, logits)

/Users/yngtodd/src/ornl/bayescache/bayescache/models/mtcnn.py:82: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  losses[key] = F.cross_entropy(F.softmax(y_pred[key]), y_true[key])


In [16]:
loss

{'subsite': tensor(1.9566, grad_fn=<NllLossBackward>),
 'laterality': tensor(0.7212, grad_fn=<NllLossBackward>),
 'histology': tensor(1.3108, grad_fn=<NllLossBackward>),
 'grade': tensor(1.1650, grad_fn=<NllLossBackward>)}